**Code**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/data.csv')
df1.head()

**Data Cleaning Step 1**

**Looking for incorrect Scans**

1. Identify the barcodes with their timestamps where they have these issues

2. Remove System Design issues

3. Drop the rows where we have above mentioned issues from our main dataframe

4. For issues with 3 rows for same timestamp in one barcode - No pattern identified


In [ ]:
temp = df.groupby(['barcode','timestamp'])['scan_to_station'].count().reset_index().sort_values(by='scan_to_station',ascending=False)
temp

temp_2 = temp[temp['scan_to_station']==2].reset_index().drop('index',axis=1)
temp_2.shape

temp_2_df = pd.DataFrame()
for i in temp_2.index:
  match = df[(df['barcode']== temp_2['barcode'].iloc[i]) & (df['timestamp']==temp_2['timestamp'].iloc[i])]
  temp_2_df = pd.concat([temp_2_df,match])

temp_2_df
temp_2_df = temp_2_df[ ~((temp_2_df['scan_from_station'] == 'Customer Receiving') & (temp_2_df['scan_to_station'] == 'Pre-Inspection')) ]
temp_2_df = temp_2_df[ ~((temp_2_df['scan_from_station'] == 'Customer') & (temp_2_df['scan_to_station'] == 'Customer Receiving')) ]
temp_2_df = temp_2_df.sort_values(by=['barcode','timestamp']).reset_index().drop('index',axis = 1)

i = 0
while i+ 1< temp_2_df.shape[0] :
  if temp_2_df['scan_from_station'].iloc[i] == temp_2_df['scan_to_station'].iloc[i+1] and temp_2_df['scan_to_station'].iloc[i] == temp_2_df['scan_from_station'].iloc[i+1]:
    df = df.drop(df[(df['barcode'] == temp_2_df['barcode'].iloc[i]) & (df['timestamp'] == temp_2_df['timestamp'].iloc[i])].index)
  i = i + 2


temp_3 = temp[temp['scan_to_station']==3].reset_index().drop('index',axis=1)
temp_3.shape

temp_3_df= pd.DataFrame()
for i in temp_3.index:
  match = df1[(df1['barcode']== temp_3['barcode'].iloc[i]) & (df1['timestamp']==temp_3['timestamp'].iloc[i])]
  temp_3_df = pd.concat([temp_3_df,match])

df.shape

**Data Cleaning/Validation Part 2**
1. Convert Timestamp from string to time
2. All values in scan_to_station are present in scan_from_station
3. Max Min and Range
4. Looking for null values and duplicate values
5. Data Sanity Checks for Time Series Data
6. Total number of products we are dealing with = Sum of products entered the system daily
7. Which products never entered the System during the time frame
8. Products which shipped > 1 during the time frame


In [ ]:
print(df.shape)
print(df.dtypes)
df['timestamp'] = pd.to_datetime(df['timestamp'])
print(type(df['timestamp'][1]))

print(set(df['scan_to_station']).issubset(set((df['scan_from_station']))))

print(min(df['timestamp']))
print(max(df['timestamp']))
print((max(df['timestamp'])) - (min(df['timestamp'])))

print(df.drop_duplicates().shape,df.shape)
nulls = df.isnull().sum()
print(nulls[nulls > 0])

df['date'] = df['timestamp'].map(lambda x: x.date())
print(df[df['scan_from_station']=='Customer'].groupby('date').agg({'barcode':'nunique'}))
print(df[df['scan_from_station']=='Customer'].groupby('date').agg({'barcode':'nunique'}).sum())

df = df.sort_values(by=['barcode','timestamp'])
df['timestamp_next'] = df['timestamp'].shift(-1)
time = []
pd.Series(df['barcode'].unique()).map(generate_time)
df['time'] = time

print(df[(df['timestamp'].map(lambda x: x.strftime("%m")).astype('int') < 1) | (df['timestamp'].map(lambda x: x.strftime("%m")).astype('int') > 12)])
print(df[(df['timestamp'].map(lambda x: x.strftime("%d")).astype('int') < 1) | (df['timestamp'].map(lambda x: x.strftime("%d")).astype('int') > 31)])
print(df[(df['timestamp'].map(lambda x: x.strftime("%H")).astype('int') < 0) | (df['timestamp'].map(lambda x: x.strftime("%m")).astype('int') > 24)])
print(df[(df['timestamp'].map(lambda x: x.strftime("%M")).astype('int') < 0) | (df['timestamp'].map(lambda x: x.strftime("%d")).astype('int') > 60)])
print(df[(df['timestamp'].map(lambda x: x.strftime("%S")).astype('int') < 0) | (df['timestamp'].map(lambda x: x.strftime("%d")).astype('int') > 60)])

print(df['barcode'].nunique())

print(df[(df['scan_to_station']=='Customer') & (df['scan_from_station']!='Ship')])

print(df[df['scan_to_station'] == 'Ship'].groupby('barcode')['timestamp'].count().reset_index().sort_values(by='timestamp',ascending=False))

**Backlog Products**

We are trying to identify products still in the system who joined the system before the week began and therefore demand higher priority 
1. Create Unique Barcodes set
2. Identify if we have backlog products
3. Find the barcode of these products (random)
4. Flag them for our dashboard

In [ ]:
print(df[df['scan_from_station']=='Customer'].groupby('date').agg({'barcode':'nunique'}))
print(df[df['scan_from_station']=='Customer'].groupby('date').agg({'barcode':'nunique'}).sum())
print(df['barcode'].unique()) 

unique_barcodes = pd.DataFrame(df['barcode'].unique())

random_products = unique_barcodes[0][~ pd.DataFrame(df['barcode'].unique())[0].isin((df[df['scan_from_station']=='Customer']['barcode']))].reset_index()[0]
random_products

start = pd.DataFrame()
end = pd.DataFrame()
for i in range(len(random_products)):
  temp_start = pd.DataFrame(df[df['barcode'] == random_products.iloc[i]].sort_values(by='timestamp').reset_index().iloc[0]).T
  temp_end = pd.DataFrame(df[df['barcode'] == random_products.iloc[i]].sort_values(by='timestamp',ascending= False).reset_index().iloc[0]).T
  start = pd.concat([start,temp_start]) 
  end = pd.concat([end,temp_end]) 

start = start.reset_index()
end = end.reset_index()
start
start.drop(['level_0','index'],axis=1,inplace=True)
end.drop(['level_0','index'],axis=1,inplace=True)

print(pd.merge(df,df.groupby(['barcode']).agg({'timestamp':'min'}), how='inner', left_on = ['barcode','timestamp'],right_on = ['barcode','timestamp']).groupby('barcode')['timestamp'].count().reset_index().sort_values(by='timestamp',ascending = False)['timestamp'].value_counts())

**5891 products out of 14402 products entered the system in the past week (backlog)**

**Questions Answered**

1. How many times each product entered that station
2. How many unique stations do we have

In [ ]:
barcode_station = pd.crosstab(df['barcode'],df['scan_from_station'])
print(barcode_station.head())

print(df['scan_to_station'].unique())

**Minimum time to finish and actual time**

**Products which were Shipped/ Went to Rack/ Retired Saleable/ Missing/ Showroom**

1. Find Latest and First Date for Unique Barcodes
2. Find Ideal Dates based on when they should have shipped with 100% efficiency
3. Add Backlog, Shipped, Rack, Missing, Retired, Missing Columns
4. Generate Total Hours, Ideal Hours and Variance 


In [ ]:
unique_barcodes['latest_date'] = unique_barcodes[0].map(last_data_point)
unique_barcodes['Ideal_last_date_QC'] = unique_barcodes['barcode'].map(ideal_last_data_point_QC)
unique_barcodes['first_date'] = unique_barcodes[0].map(first_data_point)
unique_barcodes['Ideal_first_date_QC'] = unique_barcodes['barcode'].map(ideal_first_data_point_QC)
unique_barcodes['total_days'] = unique_barcodes['latest_date']-  unique_barcodes['first_date']
unique_barcodes['Ideal_Days_QC'] = unique_barcodes['Ideal_last_date_QC']-  unique_barcodes['Ideal_first_date_QC']

ideal_dates = df.groupby(['barcode','scan_to_station','scan_from_station'])['timestamp'].min().reset_index()

ideal_barcode = ideal_dates.groupby(['barcode']).agg({'timestamp':['min','max']}).reset_index()
ideal_barcode['ideal_days'] = abs(ideal_barcode.timestamp['max'] - ideal_barcode.timestamp['min'])

unique_barcodes = pd.concat([unique_barcodes,ideal_barcode],axis = 1)
unique_barcodes.drop(('barcode', ''),axis=1,inplace=True)
unique_barcodes.columns = ['barcode','latest_date','Ideal_last_date_QC','first_date','Ideal_first_date_QC','total_days','Ideal_Days_QC','ideal_first_date','ideal_last_date','ideal_days'] 

random_barcode= pd.DataFrame(data = random_products)
random_barcode.columns = ['bar']

unique_barcodes = pd.merge(unique_barcodes,random_barcode,how='left',left_on='barcode', right_on='bar')
#print(unique_barcodes)

#Creating Backlog Column
unique_barcodes['Backlog Product'] = np.where(unique_barcodes['bar'].isnull(),0,1)
unique_barcodes.drop(['bar'],axis=1,inplace=True)

#confirm we got all backlog products
unique_barcodes['Backlog Product'].value_counts()

#checking if products shipped/rack/retired saleable/missing/showroom
unique_barcodes['Rack'] = 0
unique_barcodes['Shipped'] = 0
unique_barcodes['Ship_Times'] = 0
unique_barcodes['Retired Saleable'] = 0
unique_barcodes['Missing'] = 0
unique_barcodes['Showroom'] = 0
unique_barcodes['Spotting'] = 0
unique_barcodes['Pressing'] = 0
unique_barcodes['Cleaning'] = 0
unique_barcodes['Quality Control'] = 0
unique_barcodes['Evaluate Product'] = 0


for i in range(len(unique_barcodes)):
  unique_barcodes['Shipped'][i] = 'Ship' in ' '.join(df[df['barcode'] == unique_barcodes['barcode'].iloc[i]]['scan_to_station']) + ' '.join(df[df['barcode'] == unique_barcodes['barcode'].iloc[i]]['scan_from_station'])
  unique_barcodes['Evaluate Product'][i] = 'Evaluate Product' in ' '.join(df[df['barcode'] == unique_barcodes['barcode'].iloc[i]]['scan_to_station']) + ' '.join(df[df['barcode'] == unique_barcodes['barcode'].iloc[i]]['scan_from_station'])
  unique_barcodes['Ship_Times'][i] = df[(df['barcode'] == unique_barcodes['barcode'].iloc[i]) & (df['scan_to_station'] == 'Ship')]['scan_to_station'].count()
  unique_barcodes['Rack'][i] = 'Rack' in ' '.join(df[df['barcode'] == unique_barcodes['barcode'].iloc[i]]['scan_to_station']) +  ' '.join(df[df['barcode'] == unique_barcodes['barcode'].iloc[1]]['scan_from_station'])
  unique_barcodes['Showroom'][i] = 'Showroom Staging' in ' '.join(df[df['barcode'] == unique_barcodes['barcode'].iloc[i]]['scan_to_station']) + ' '.join(df[df['barcode'] == unique_barcodes['barcode'].iloc[i]]['scan_from_station'])
  unique_barcodes['Missing'][i] = 'Missing' in ' '.join(df[df['barcode'] == unique_barcodes['barcode'].iloc[i]]['scan_to_station']) +  ' '.join(df[df['barcode'] == unique_barcodes['barcode'].iloc[1]]['scan_from_station'])
  unique_barcodes['Retired Saleable'][i] = 'Retired Saleable' in ' '.join(df[df['barcode'] == unique_barcodes['barcode'].iloc[i]]['scan_to_station']) +  ' '.join(df[df['barcode'] == unique_barcodes['barcode'].iloc[1]]['scan_from_station'])
  unique_barcodes['Spotting'][i] = df[(df['barcode'] == unique_barcodes['barcode'].iloc[i]) & (df['scan_to_station'] == 'Spotting')]['scan_to_station'].count()
  unique_barcodes['Cleaning'][i] = df[(df['barcode'] == unique_barcodes['barcode'].iloc[i]) & (df['scan_to_station'] == 'Cleaning')]['scan_to_station'].count()
  unique_barcodes['Pressing'][i] = df[(df['barcode'] == unique_barcodes['barcode'].iloc[i]) & (df['scan_to_station'] == 'Pressing')]['scan_to_station'].count()
  unique_barcodes['Quality Control'][i] = df[(df['barcode'] == unique_barcodes['barcode'].iloc[i]) & (df['scan_to_station'] == 'Quality Control')]['scan_to_station'].count()


unique_barcodes['total_hours'] = round(unique_barcodes['total_days'].map(lambda x: x.days) * 24 + (unique_barcodes['total_days'].map(lambda x: x.seconds)/60)/60,2)
unique_barcodes['ideal_hours'] = round(unique_barcodes['ideal_days'].map(lambda x: x.days) * 24 + (unique_barcodes['ideal_days'].map(lambda x: x.seconds)/60)/60,2)
unique_barcodes['Variance'] = round(unique_barcodes['total_hours'] - unique_barcodes['ideal_hours'],2)

print(unique_barcodes)

**Top 10 worse products for all systems daily**

In [ ]:
worse_products = pd.DataFrame()
for i in (set(df['scan_to_station'])):
  #print(i)
  for j in set(df['date']):
    temp = pd.DataFrame(df[(df['scan_from_station'] == i) & (df['date'] == j)].sort_values(by='time',ascending= False)['barcode'].head(10).reset_index().drop(['index'],axis = 1,inplace = False))
    temp['process'] = i
    temp['date'] = j
    worse_products = pd.concat([worse_products,temp])

print(worse_products)
#worse_products.to_csv('Worse_Products.csv')


df = pd.merge(df,worse_products,how='left',left_on=['barcode','date','scan_to_station'],right_on=['barcode','date','process'])


**Understanding different segments for Products**

In [ ]:
unique_barcodes.groupby(['Shipped','Rack','Backlog Product'])['barcode'].count()

**Export Data**

In [ ]:
unique_barcodes.to_csv('unique_barcodes.csv')
df.to_csv('Inbound_Process.csv')
worse_products.to_csv('Worse_Products.csv')

**Funcations Used**


In [ ]:
def generate_time(bar):
  temp = df[df['barcode'] == bar].sort_values(by='timestamp')['timestamp'].diff()
  return time.extend(temp.map(lambda x: round((x.total_seconds() / 60),2)))


def last_data_point(bar):
    #print(bar)
    temp = df[df['barcode'] == bar].sort_values(by='timestamp',ascending = False)['timestamp']
    return temp.iloc[0]

def ideal_first_data_point_QC(bar):
    #print(bar)
    temp = df[df['barcode'] == bar].sort_values(by='timestamp')
    try:
      return temp[temp['scan_to_station']=='Quality Control']['timestamp'].iloc[0]
    except:
      return temp['timestamp'].iloc[0]

def ideal_last_data_point_QC(bar):
    #print(bar)
    temp = df[df['barcode'] == bar].sort_values(by='timestamp',ascending=False)
    try:
      return temp[temp['scan_to_station']=='Quality Control']['timestamp'].iloc[0]
    except:
      return temp['timestamp'].iloc[0]


def first_data_point(bar):
    #print(bar)
    temp = df[df['barcode'] == bar].sort_values(by='timestamp')['timestamp']
    return temp.iloc[0]

def set_values(df,df2):
  for i in df2:
    i.isin(df)